In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
# Import needed packages
import tensorflow as tf
from tensorflow import keras
from keras.models import Model, load_model
from keras.preprocessing.image import ImageDataGenerator, DirectoryIterator
from keras.applications.resnet import preprocess_input as pre_res
from keras.applications.densenet import preprocess_input as pre_dense
from sklearn.metrics import confusion_matrix, plot_confusion_matrix
from keras import optimizers

In [3]:
main_path = '/content/gdrive/My Drive/Thesis/Data/COVID-classification/'
test_path_org = main_path+'test_new/'
test_path_mask = main_path+'Ex_test/'

In [4]:
model_path = '/content/gdrive/My Drive/Thesis/models/'

with tf.device('/device:GPU:0'):
  ResNet = load_model(model_path+'ResNet-1.h5', compile=False)
  ResNet_mask = load_model(model_path+'ResNet_masks-1.h5', compile=False)
  DenseNet = load_model(model_path+'DenseNet-1.h5', compile=False)
  DenseNet_mask = load_model(model_path+'DenseNet_masks-1.h5', compile=False)

In [5]:
model = DenseNet_mask

In [6]:
datagen = ImageDataGenerator(
    preprocessing_function=pre_dense)

In [7]:
test_set = DirectoryIterator(test_path_org,
                            datagen,
                            target_size = (512, 512),
                            batch_size = 32,
                            class_mode = 'binary',
                            shuffle = False)

Found 3794 images belonging to 2 classes.


In [8]:
test_set_mask = DirectoryIterator(test_path_mask,
                            datagen,
                            target_size = (512, 512),
                            batch_size = 32,
                            class_mode = 'binary',
                            shuffle = False)

Found 3794 images belonging to 2 classes.


In [9]:
with tf.device('/device:GPU:0'):
  y_pred = model.predict(test_set_mask)

In [17]:
confusion_matrix(test_set_mask.labels, y_pred.round())

array([[1755,  193],
       [1629,  217]])

In [11]:
adam = optimizers.Adam()
model.compile(loss='binary_crossentropy',
              optimizer=adam,
              metrics=['accuracy'])

In [12]:
# original data
with tf.device('/device:GPU:0'):
  model.evaluate(test_set)

119/119 [==============================] - 2701s 22s/step - loss: 0.8844 - accuracy: 0.7922


In [15]:
with tf.device('/device:GPU:0'):
  model.evaluate(test_set_mask)

119/119 [==============================] - 2690s 23s/step - loss: 1.6867 - accuracy: 0.5198
